# All events analysis

Perform the gene-level GSEA analysis, for all types of AS events

In [1]:
from datetime import datetime; print("START:", datetime.now())
import socket; print("Simons Foundation, Rusty HPC,", socket.gethostname())

START: 2021-10-24 21:50:49.182622
Simons Foundation, Rusty HPC, worker3124


In [2]:
%cd /mnt/home/zzhang/ceph/CHARM-AlternativeSplicing
%load_ext rpy2.ipython

/mnt/ceph/users/zzhang/CHARM-AlternativeSplicing


## 1. Read in Data

In [3]:
from jemm.meta_loader import MetaLoader
from jemm.genomic_annotation import ExonSet
from jemm.plots import volcano_plot
from jemm.covariate import Contrasts, Covariate
from jemm.utils import fdr_bh
from collections import Counter
import scipy.stats
import seaborn as sns
import gseapy as gp
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
plt.style.use(['science', 'no-latex', 'ieee'])


DATA_VER = 'data-V7'
PCS_TO_INCL = '0,6'
USE_RE = True

%run ./notebook/navy_utils.V7.py $DATA_VER $PCS_TO_INCL $USE_RE

FIGDIR = "%s/all_genes_splicing/figs" % DATA_VER
os.makedirs(FIGDIR, exist_ok=True)

contrast_cols = ['final@Pre', 'final@First', 'final@Mid', 'final@Post', 
                 'final@False Negative','final@Immune', 
                 #'final@Reinfection',
                 #'final@Asymptomatic', 
                 #'final@Exposed', 
                 #'final@Mild',
                 #'final@Moderate'
                ]

navy_utils.py data-V7 ['PC0', 'PC6'] True


In [ ]:
metaloader = MetaLoader(covs, joint_data_files, joint_reg_tables,
                        jem_type='lmm' if USE_RE else 'lm',
                        jem_kwargs={'diff_intercept_by_measure':True,
                            'group_varname': 'pid'})

loading SE..
loaded n=(81341, 1444) jct, n=(47847, 1444) txr


/mnt/ceph/users/zzhang/RnaSplicingHub/jemm/jemm/model.py:176: UserWarning: Input covariate matrix has condition number = 39865.375010374875 > 100, indicating the presence of co-linearality; optimization might be unstable.
  _ = self._check_covariate_condition()
/mnt/home/zzhang/workspace/programs/anaconda3/envs/rna-seq/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


loading A5SS..
loaded n=(3705, 1176) jct, n=(17687, 1444) txr


/mnt/ceph/users/zzhang/RnaSplicingHub/jemm/jemm/model.py:176: UserWarning: Input covariate matrix has condition number = 39865.375010374875 > 100, indicating the presence of co-linearality; optimization might be unstable.
  _ = self._check_covariate_condition()
/mnt/home/zzhang/workspace/programs/anaconda3/envs/rna-seq/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


loading A3SS..


In [ ]:
gene_centric_dict = metaloader.get_gene_centric_dict(
    cols=contrast_cols,
    fdr_thresh=0.05
)

## 2. Get GO enrichment for each condition

In [ ]:
gobp_dfs = {}
gocc_dfs = {}

wd = "./%s/all_genes_splicing/" % DATA_VER
union_genes = []
for cond in contrast_cols:
    genes = [g for g in gene_centric_dict if cond in gene_centric_dict[g] if type(g) is str]
    union_genes.extend(genes)
    print("%s: %i"%(cond, len(genes)))
    with open(os.path.join(wd, cond, "genes.txt"), 'w') as f:
        f.write('\n'.join(genes))
    # GOBP
    this_wd = os.path.join(wd, cond, "GOBP")
    os.makedirs(this_wd, exist_ok=True)
    _ = gp.enrichr(gene_list=list(genes), description='pathway', gene_sets='GO_Biological_Process_2018',
                   outdir=this_wd,
                   format="png",
                   cutoff=0.1,
                  )
    gobp_dfs[cond] = pd.read_table(os.path.join(this_wd, "GO_Biological_Process_2018.human.enrichr.reports.txt"))
    # GOCC
    this_wd = os.path.join(wd, cond, "GOCC")
    os.makedirs(this_wd, exist_ok=True)
    _ = gp.enrichr(gene_list=list(genes), description='pathway', gene_sets='GO_Cellular_Component_2018',
                   outdir=this_wd,
                   format="png",
                   cutoff=0.1,
                  )
    gocc_dfs[cond] = pd.read_table(os.path.join(this_wd, "GO_Cellular_Component_2018.human.enrichr.reports.txt"))

union_genes = list(set(union_genes))
with open('%s/union_das_genes.txt'%wd, 'w') as f:
    f.write('\n'.join(union_genes))
print('union genes=%i' % len(union_genes))

In [ ]:
# UNION GO ENRICHR
cond = 'UNION'
this_wd = os.path.join(wd, cond, "GOBP")
os.makedirs(this_wd, exist_ok=True)
_ = gp.enrichr(gene_list=list(union_genes), description='pathway', gene_sets='GO_Biological_Process_2018',
               outdir=this_wd,
               format="png",
               cutoff=0.1,
              )
gobp_dfs[cond] = pd.read_table(os.path.join(this_wd, "GO_Biological_Process_2018.human.enrichr.reports.txt"))

# GOCC
this_wd = os.path.join(wd, cond, "GOCC")
os.makedirs(this_wd, exist_ok=True)
_ = gp.enrichr(gene_list=list(union_genes), description='pathway', gene_sets='GO_Cellular_Component_2018',
               outdir=this_wd,
               format="png",
               cutoff=0.1,
              )
gocc_dfs[cond] = pd.read_table(os.path.join(this_wd, "GO_Cellular_Component_2018.human.enrichr.reports.txt"))

In [ ]:
(gobp_dfs['UNION']['Adjusted P-value'] < 0.05).sum()

In [ ]:
# Plot the number of genes with *any* DAS along the disease progression
cond_to_das_genes = {}
for cond in contrast_cols:
    genes = [g for g in gene_centric_dict if cond in gene_centric_dict[g]]
    cond_to_das_genes[cond] = len(genes)

cond_to_das_genes = pd.Series(cond_to_das_genes, index=contrast_cols)
cond_to_das_genes.loc[
    ['final@Pre',
     'final@First',
     'final@Mid',
     'final@Post'
    ]
].plot(kind='bar')
plt.savefig('%s/01-num_das_genes.pdf' % FIGDIR)

In [ ]:
# write GO for ReviGO server
import re
def write_revigo(df, fp, fdr_cutoff=0.01):
    subdf = df.loc[df["Adjusted P-value"] < fdr_cutoff, ['Term', 'Adjusted P-value']].copy()
    subdf['Term'] = [x.split()[-1].strip('()') for x in subdf['Term']]
    subdf = subdf.sort_values('Adjusted P-value', ascending=True)
    subdf.to_csv(fp, sep="\t", index=False)
    return subdf
    
k = 'final@First'
df = gobp_dfs[k]
df = write_revigo(df, fp='./%s/all_genes_splicing/final@First/for_revigo.fdr0.05.txt' % DATA_VER, fdr_cutoff=0.05)
df = write_revigo(df, fp='./%s/all_genes_splicing/final@First/for_revigo.fdr0.01.txt' % DATA_VER, fdr_cutoff=0.01)

In [ ]:
k = 'final@Mid'
df = gobp_dfs[k]
df = write_revigo(df, fp='./%s/all_genes_splicing/final@Mid/for_revigo.fdr0.05.txt' % DATA_VER, fdr_cutoff=0.05)
df = write_revigo(df, fp='./%s/all_genes_splicing/final@Mid/for_revigo.fdr0.01.txt' % DATA_VER, fdr_cutoff=0.01)

In [ ]:
k = 'UNION'
df = gobp_dfs[k]
df = write_revigo(df, fp='./%s/all_genes_splicing/UNION/for_revigo.fdr0.05.txt' % DATA_VER, fdr_cutoff=0.05)
df = write_revigo(df, fp='./%s/all_genes_splicing/UNION/for_revigo.fdr0.01.txt' % DATA_VER, fdr_cutoff=0.01)

## 3. Temporary change of GO terms 

In [ ]:
union_df = gobp_dfs['UNION']
union_df = union_df.loc[union_df['Adjusted P-value']<0.9]
total_union_go_terms = [x for x in union_df['Term']]

In [ ]:
gobp_dfs.keys()

In [ ]:
temporal_conds = ['final@Pre', 'final@First', 'final@Mid', 'final@Post']
union_go = np.zeros((len(total_union_go_terms), len(temporal_conds)))

def compute_overlap(s):
    ol, tot = s.split('/')
    ol = float(ol)
    tot = float(tot)
    p = ol / tot * 100
    std = np.sqrt(p * (1-p) / tot)
    return p, std

for i, cond in enumerate(temporal_conds):
    foo = gobp_dfs[cond].copy()
    foo.index = foo['Term']
    union_go[:, i] = [compute_overlap(foo.loc[k, 'Overlap'])[0] if k in foo.index else 0 for k in total_union_go_terms]

union_go_df = pd.DataFrame(union_go, index=total_union_go_terms, columns=[x.split('@')[1] for x in temporal_conds])

In [ ]:
union_go_df = union_go_df.replace(0, np.nan)

In [ ]:
target_go = [
    'neutrophil degranulation (GO:0043312)',
    'SRP-dependent cotranslational protein targeting to membrane (GO:0006614)',
    'regulation of alternative mRNA splicing, via spliceosome (GO:0000381)',
    'viral process (GO:0016032)',
    #'intracellular protein transport (GO:0006886)',
    #'mRNA processing (GO:0006397)',
    #'ribonucleoprotein complex assembly (GO:0022618)',
    #'androgen receptor signaling pathway (GO:0030521)',
    #'intracellular estrogen receptor signaling pathway (GO:0030520)',
    #'positive regulation of type I interferon production (GO:0032481)',
    #'RNA secondary structure unwinding (GO:0010501)',
    #'glycolipid biosynthetic process (GO:0009247)'
    #'protein targeting to vacuole (GO:0006623)'
    #'antigen processing and presentation of exogenous peptide antigen (GO:0002478)',
]

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
union_go_df.loc[target_go].transpose().plot(ax=ax, marker='o')
ax.legend(bbox_to_anchor=(-0.15, 0.75), loc='upper right')
ax.set_ylabel('% of GO Term genes with DAS')
fig.savefig('%s/02-GO_temp_changes.pdf' % FIGDIR)

In [ ]:
# export genes with DAS on AS regulation
first_sf_gen = gobp_dfs['final@First'].query('Term=="regulation of alternative mRNA splicing, via spliceosome (GO:0000381)"').Genes.to_list()[0].split(';')
mid_sf_gen = gobp_dfs['final@Mid'].query('Term=="regulation of alternative mRNA splicing, via spliceosome (GO:0000381)"').Genes.to_list()[0].split(';')
sf_gen = list(set(first_sf_gen + mid_sf_gen))

In [ ]:
sf_gen_df = pd.DataFrame(columns=['event_id', 'event_type', 'gene', 'condition', 'base_psi', 'delta_psi'])
for gen in sf_gen:
    for cond in gene_centric_dict[gen]:
        for evt in gene_centric_dict[gen][cond]:
            this = {'event_id': evt.event_id, 
                    'event_type': evt.event_type,
                    'gene': gen,
                    'condition': cond,
                    'base_psi': evt._base_psi,
                    'delta_psi': evt._delta_psi
                   }
            sf_gen_df = sf_gen_df.append(this, ignore_index=True)

In [ ]:
sf_gen_df.sort_values('condition').to_csv('%s/all_genes_splicing/SplicingFactor_DAS.tsv' % DATA_VER, index=False, sep="\t")
sf_gen_df.sort_values('condition')

## 4. Enrichment of CLIP and Pfam annotations

## 4.1 CLIP

In [ ]:
das_eids = {
    as_type: [eid for eid in metaloader.data[as_type].stats_tests if \
              (metaloader.data[as_type].stats_tests[eid].loc[contrast_cols, 'qvals']<0.05).sum() > 0]
    for as_type in ('A5SS', 'A3SS', 'RI', 'SE')
}

In [ ]:
def plot_volcano(das_eids, ax=None):
    clip_test = ExonSet.enrichment_test(das_eids, metaloader, 'rbp_clip')
    # only look at ENCODE IDR peaks (?)
    clip_test = clip_test.loc[[i for i in clip_test.index if clip_test.loc[i, 'items'].endswith('IDR')]]
    # vocalno plot
    clip_test['log2_ratios'] = np.log2(clip_test['ratios'])
    clip_test['neglog10_pvals'] = -np.log10(clip_test['pvals'])
    clip_test = clip_test.replace([-np.inf, np.inf], np.nan).dropna(axis=0)
    tot_fg = sum([len(das_eids[x]) for x in das_eids])
    #print('total foreground = %i' % tot_fg)
    #clip_test = clip_test.query('observed > %f' % (tot_fg*0.01))
    clip_test = clip_test.query('observed > 50')
    clip_test['fdr'] = fdr_bh(clip_test['pvals'])
    clip_test['labels'] = ''
    clip_test.at[clip_test.query('fdr<0.05').sort_values('fdr', ascending=True).head(3).index, 'labels'] = \
        clip_test['items'].apply(lambda x: x.split('_')[0])
    clip_test.at[clip_test.query('fdr<0.05 and ratios>2').sort_values('fdr', ascending=True).head(3).index, 'labels'] = \
        clip_test['items'].apply(lambda x: x.split('_')[0])
    ax = volcano_plot(
        x='log2_ratios', 
        y='neglog10_pvals', 
        xcut=1,
        ycut=clip_test.query('fdr<0.05').sort_values('neglog10_pvals', ascending=True)['neglog10_pvals'].iloc[0],
        labels='labels', 
        data=clip_test,
        ax=ax
    )
    ax.set_xlabel('log2(Ratio)')
    ax.set_ylabel('log10(Pvalue)')
    return clip_test, ax

In [ ]:
clip_test, ax = plot_volcano(das_eids)
plt.savefig('%s/03-clip_test.pdf'% FIGDIR)
plt.show()

In [ ]:
clip_test.query('fdr < 0.05 and ratios>=2').sort_values('fdr', ascending=True)

## 4.2 Pfam

In [ ]:
pfam_test = ExonSet.enrichment_test(das_eids, metaloader, 'pfam_domain')
pfam_test = pfam_test.query('observed > 5')

In [ ]:
# vocalno plot
pfam_test['log2_ratios'] = np.log2(pfam_test['ratios'])
pfam_test['neglog10_pvals'] = -np.log10(pfam_test['pvals'])
pfam_test['fdr'] = fdr_bh(pfam_test['pvals'])
pfam_test['labels'] = ''
pfam_test.at[pfam_test.query('fdr<0.05').sort_values('fdr', ascending=True).head(6).index, 'labels'] = \
    pfam_test['items']
ax = volcano_plot(
    x='log2_ratios', 
    y='neglog10_pvals', 
    xcut=1,
    ycut=pfam_test.query('fdr<0.05').sort_values('neglog10_pvals', ascending=True)['neglog10_pvals'].iloc[0],
    labels='labels', 
    data=pfam_test)
ax.set_xlabel('log2(Ratio)')
ax.set_ylabel('log10(Pvalue)')
plt.savefig('%s/03-pfam_test.pdf'% FIGDIR)

In [ ]:
pfam_test.query('fdr<0.05').sort_values('fdr', ascending=True)

In [ ]:
print("FINISH:", datetime.now())